In [1]:
import boto3
import pandas as pd

In [2]:
df = pd.read_csv('C:\\Users\\yugant.shekhar\\OneDrive - Blue Altair\\Desktop\\Douments\\AWS\\temp.csv')

# print(df['Access key ID'][0])

In [3]:
aws_access_key = df['Access key ID'][0]
aws_secret_key = df['Secret access key'][0]

In [4]:
s3 = boto3.resource(
    service_name = 's3'
    , region_name = 'eu-north-1'
    , aws_access_key_id=aws_access_key
    , aws_secret_access_key=aws_secret_key
)

In [5]:
for bucket in s3.buckets.all():

    print(bucket)

s3.Bucket(name='de-retail-bucket')


In [44]:
bucket = 'de-retail-bucket'

In [6]:
# Upload to S3
# CSV
s3.Bucket('de-retail-bucket').upload_file(Filename='./dim_customer.csv', Key='sales_data/dim_customer3.csv')

In [53]:
# List files in a folder
# for finding .parquest file
import os 

# Specify the folder path
folder_path = './customer_data_mart'

# List all files in the folder
file_names = os.listdir(folder_path)

# Optionally, filter out only files (ignore directories)
file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]

for i in file_names:

    # print(i.split('.')[-1])
    if 'parquet' == i.split('.')[-1]:
        original_file = print(i)
       
        

# Print the file names
# print(file_names)

part-00000-c6ced50b-10ce-4bab-9fb6-7a8b675a0e0a-c000.snappy.parquet


In [45]:
# Upload to S3
# Parquet

s3.Bucket(bucket).upload_file(Filename= f'./customer_data_mart/{original_file}', Key='sales_data/customer_data_mart/customer_data_mart.parquet')

In [6]:
# Read Parquet
import io

obj = s3.Bucket('de-retail-bucket').Object('sales_data/customer_data_mart/customer_data_mart.parquet').get()

buffer = io.BytesIO(obj['Body'].read())

customer_df = pd.read_parquet(buffer)
customer_df.head()


,customer_id,first_name,last_name,address,pincode,phone_number,sales_date,total_cost
0,1637,Anita,Barman,"H.No. 88, Upadhyay Chowk, Orai 493235",493235,29562838,2020-10-24,4543
1,6317,Riya,Bassi,"23/159, Parmar Zila, Sri Ganganagar-469379",469379,0,2022-01-01,4301
2,2420,Raghav,Ramaswamy,"H.No. 24, Memon Path, Nagercoil 265985",265985,0,2021-10-07,3584
3,7350,Gautam,Palla,"H.No. 31, Kashyap, Bally 172013",172013,0,2020-01-04,3220
4,7350,Gautam,Palla,"H.No. 31, Kashyap, Bally 172013",172013,0,2020-01-04,3220


In [7]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
            .appName('test AWS')\
            .getOrCreate()


# obj = s3.Bucket('de-retail-bucket').Object('sales_data/customer_data_mart/customer_data_mart.parquet').get()

# buffer = io.BytesIO(obj['Body'].read())

# df = spark.read.format('parquet')\
#     .option('header', 'true')\
#     .option('inferSchema', 'true')\
#     .load(buffer)

df = spark.createDataFrame(customer_df)

df.show()

Py4JJavaError: An error occurred while calling o46.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [54]:
# List Objects
for obj in bucket.objects.filter(Prefix='sales_data/'):
    print(obj.key)

AttributeError: 'str' object has no attribute 'objects'

In [31]:
# Read from S3
import pandas as pd

obj = s3.Bucket('de-retail-bucket').Object('sales_data/dim_customer.csv').get()
customer_df = pd.read_csv(obj['Body'])
customer_df.head()

,customer_id,first_name,last_name,address,pincode,phone_number,customer_joining_date
0,4986,Balveer,Choudhury,"31/30, Devan Nagar, Anand 799736",376853,913000656779,2021-04-20
1,6642,Kalpit,Divan,"15, Zacharia Nagar, Buxar-919980",331701,911454391537,2022-05-14
2,7763,Dominic,Radhakrishnan,"834, Rajagopal Road, Amaravati 801199",351147,1346045138,2022-02-28
3,2420,Raghav,Ramaswamy,"H.No. 24, Memon Path, Nagercoil 265985",668002,6956574158,2022-01-21
4,8576,Zinal,Venkataraman,"832, Majumdar Road, Ongole-286732",705304,3399052109,2022-07-20


In [89]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
            .appName('silver_layer')\
            .getOrCreate()

In [90]:
import logging

# clean_path = '''C:\\Users\\yugant.shekhar\\OneDrive - Blue Altair\\Desktop\\Douments\\Spark\\Retail Project\\Data\\actual_data\\cleaned'''
logger = logging.getLogger(__name__)

In [91]:
from s3_read_write import S3

ob = S3(logger)

# customer_df = ob.read_file(spark, 'sales_data/customer_data_mart/customer_data_mart.parquet', 'parquet')

# ob.upload_to_s3('trial_upload/customer_data_mart.parquet', './customer_data_mart/', is_parquet=True)

In [88]:
spark.stop()